In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
import random, json, argparse, re, os, sys
sys.path.append("../clevr_control/scripts")
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"  
from collections import defaultdict
from tqdm import tqdm
import pytz
from datetime import datetime
timezone = pytz.timezone('America/New_York') 
from helper import *

/home/yingshac/workspace/2023/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = LlamaTokenizer.from_pretrained("/data/yingshac/LLM/vicuna-7b")
model = LlamaForCausalLM.from_pretrained("/data/yingshac/LLM/vicuna-7b")

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.80s/it]


In [3]:
model.cuda()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [4]:
with open("/home/yingshac/workspace/2023/MathQA/feed_vicuna/zero_shot.jsonl", "r") as f:
    prompts = [l.strip() for l in f.readlines()]

In [4]:
verbs = [l.strip() for l in open("/home/yingshac/workspace/2023/clevr_control/feed_decoder_LM/verbs.txt", "r").readlines()]
complementary_verbs = [l.strip() for l in open("/home/yingshac/workspace/2023/clevr_control/feed_decoder_LM/complementary_transitive_verbs.txt", "r").readlines()]
print(len(verbs), len(complementary_verbs))

364 725


In [41]:
template1 = ("Question: The verb '{verb}' entails which of the following information about the spatial relationship between the subject and the object?\n\n"
      "Options:\n"
      "A. The subject is at the bottom of the object\n" # The subject is on the top and the object is at the bottom\n"
      "B. The subject is on top of the object\n" #The subject is at the bottom and the object is on the top\n"
      #"C. The verb '{verb}' entails a spatial relationship other than option A or option B\n"
      #"D. The verb '{verb}' does not entail a specific spatial relationship between the subject and the object\n"
      "\n"
      "Answer: The correct option is"
     )
template2 = (
      "Suppose X {verb_third_singular} Y. Question: X is at the bottom of Y, yes or no?\n"
      "Answer:"
)
template3 = (
      "Suppose X {verb_third_singular} Y. Question: X is on top of Y, true or false?\n"
      "Answer:"
)
template4 = (
      "Given X {verb_third_singular} Y. Question: Should we assume X is on top of Y?\n"
      "Answer:"
)
template5 = (
      "Given X {verb_third_singular} Y. Question: Does this imply that X is on top of Y?\n?"
      "Answer:"
)
template6 = (
      "If you were to generate an image according to the caption 'a red cube {verb_third_singular} a blue cube', "
      "would you put the red or the blue cube at the bottom?\n"
      "Answer:"
)
template7 = (
      "Question: An image corresponding to the caption 'a red cube {verb_third_singular} a blue cube' probably has which object on the top?"
      "Answer:"
)
template = template6
print(template.format(**{"verb_third_singular": "beats"}))

If you were to generate an image according to the caption 'a red cube beats a blue cube', would you put the red or the blue cube at the bottom?
Answer:


In [8]:
def create_in_context_ex(template, template_arg, gth_completion, ex_separator="\n\n"):
    try:
        return template.format(**template_arg) + gth_completion + ex_separator
    except KeyError:
        print("The question template and template args doesn't match!")
        raise

In [ ]:
"""
examples = [
    ("cover", "B"),
    ("press", "B"),
    ("ride", "B"),
    ("overlay", "B"),
    ("lock", "B"),
    ("dangle", "A"),
    ("hide", "A"),
    ("underpin", "A"),
    ("submerge", "A"),
    ("support", "A"),

    #("bake", "C"),
    #("chase", "C"),
    #("throw", "C"),
    #("use", "D"),
    #("specify", "D"),
    #("dream", "D")
]

examples = [
    ("cover", "Yes"),
    ("press", "Yes"),
    ("ride", "Yes"),
    ("overlay", "Yes"),
    ("lock", "Yes"),
    ("dangle", "No"),
    ("hide", "No"),
    ("underpin", "No"),
    ("submerge", "No"),
    ("support", "No"),
]

examples = [
    ("cover", "True"),
    ("press", "True"),
    ("ride", "True"),
    ("overlay", "True"),
    ("lock", "True"),
    ("dangle", "False"),
    ("hide", "False"),
    ("underpin", "False"),
    ("submerge", "False"),
    ("support", "False"),
]

examples = [
    ("cover", "Yes, we should"),
    ("press", "Yes, we should"),
    ("ride", "Yes, we should"),
    ("overlay", "Yes, we should"),
    ("lock", "Yes, we should"),
    ("dangle", "No, we should not"),
    ("hide", "No, we should not"),
    ("underpin", "No, we should not"),
    ("submerge", "No, we should not"),
    ("support", "No, we should not"),
]

examples = [
    ("cover", "Yes, it does"),
    ("press", "Yes, it does"),
    ("ride", "Yes, it does"),
    ("overlay", "Yes, it does"),
    ("lock", "Yes, it does"),
    ("dangle", "No, it does not"),
    ("hide", "No, it does not"),
    ("underpin", "No, it does not"),
    ("submerge", "No, it does not"),
    ("support", "No, it does not"),
]

examples = [
    ("cover", "The blue cube"),
    ("press", "The blue cube"),
    ("ride", "The blue cube"),
    ("overlay", "The blue cube"),
    ("lock", "The blue cube"),
    ("dangle", "The red cube"),
    ("hide", "The red cube"),
    ("underpin", "The red cube"),
    ("submerge", "The red cube"),
    ("support", "The red cube"),
    ("bake", "Unclear"),
    ("chase", "Unclear"),
    ("use", "Unclear"),
    ("specify", "Unclear"),
    ("dream", "Unclear")
]

examples = [
    ("cover", "The blue cube"),
    ("press", "The blue cube"),
    ("overlay", "The blue cube"),
    ("dangle", "The red cube"),
    ("submerge", "The red cube"),
    ("support", "The red cube"),
    ("bake", "Unclear"),
    ("use", "Unclear"),
    ("specify", "Unclear"),
    ("dream", "Unclear")
]
"""

In [49]:
# Create 10 in-context examples
examples = [
    ("cover", "The blue cube"),
    ("press", "The blue cube"),
    ("ride", "The blue cube"),
    ("overlay", "The blue cube"),
    ("lock", "The blue cube"),
    ("dangle", "The red cube"),
    ("hide", "The red cube"),
    ("underpin", "The red cube"),
    ("submerge", "The red cube"),
    ("support", "The red cube"),
    ("bake", "Unclear"),
    ("chase", "Unclear"),
    ("use", "Unclear"),
    ("specify", "Unclear"),
    ("dream", "Unclear")
]
random.shuffle(examples)

in_context_ex = ""
for ex in examples:
    in_context_ex += create_in_context_ex(
        template,
        {"verb_third_singular": convert_to_third_person_singular(ex[0])},
        f" {ex[1]}."
    )
print(in_context_ex)


If you were to generate an image according to the caption 'a red cube rides a blue cube', would you put the red or the blue cube at the bottom?
Answer: The blue cube.

If you were to generate an image according to the caption 'a red cube chases a blue cube', would you put the red or the blue cube at the bottom?
Answer: Unclear.

If you were to generate an image according to the caption 'a red cube bakes a blue cube', would you put the red or the blue cube at the bottom?
Answer: Unclear.

If you were to generate an image according to the caption 'a red cube specifies a blue cube', would you put the red or the blue cube at the bottom?
Answer: Unclear.

If you were to generate an image according to the caption 'a red cube covers a blue cube', would you put the red or the blue cube at the bottom?
Answer: The blue cube.

If you were to generate an image according to the caption 'a red cube presses a blue cube', would you put the red or the blue cube at the bottom?
Answer: The blue cube.

If

In [34]:
v = random.choice(verbs)
prompt = in_context_ex + template.format(**{"verb_third_singular": convert_to_third_person_singular(v)})
#print(prompt)
print(v)

input_ids = tokenizer([prompt]).input_ids
output_ids = model.generate(
    torch.as_tensor(input_ids).cuda(),
    do_sample=False, # do_sample=False == Greedy
    #temperature=0.7,
    max_new_tokens=4,
)
output_ids = output_ids[0][len(input_ids[0]) :]
outputs = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

present


In [35]:
print(outputs)
match = re.match(r".*(?P<output>(blue cube|red cube|Unclear)).*\.", outputs)
match.group("output")

The red cube.


'red cube'

In [50]:
### run model.generate across all verbs
all_outputs = defaultdict(list)
for v in tqdm(verbs):# + complementary_verbs):
    prompt = in_context_ex + template.format(**{"verb_third_singular": convert_to_third_person_singular(v)})

    input_ids = tokenizer([prompt]).input_ids
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        do_sample=False, # do_sample=False == Greedy
        #temperature=0.7,
        max_new_tokens=4,
    )
    output_ids = output_ids[0][len(input_ids[0]) :]
    outputs = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    #match = re.match(r"(?P<output>[A-B])\.", outputs)
    match = re.match(r".*(?P<output>(blue cube|red cube|Unclear)).*\.", outputs)

    if match is None: all_outputs["unmatched"].append(v)
    else: all_outputs[match.group("output")].append(v)


100%|██████████| 364/364 [06:34<00:00,  1.08s/it]


In [51]:
for k, v in all_outputs.items():
    print(k, len(v))

red cube 301
Unclear 26
blue cube 37


In [52]:
date = datetime.now(timezone).strftime("%m%d_%H%M%S")
print(date)
json.dump(all_outputs, open(f"../clevr_control/output_decoder_LM/vicuna7b_{date}.json", "w"), indent=4)

0905_202948


In [47]:
print(in_context_ex)

If you were to generate an image according to the caption 'a red cube uses a blue cube', would you put the red or the blue cube at the bottom?
Answer: Unclear.

If you were to generate an image according to the caption 'a red cube underpins a blue cube', would you put the red or the blue cube at the bottom?
Answer: The red cube.

If you were to generate an image according to the caption 'a red cube bakes a blue cube', would you put the red or the blue cube at the bottom?
Answer: Unclear.

If you were to generate an image according to the caption 'a red cube submerges a blue cube', would you put the red or the blue cube at the bottom?
Answer: The red cube.

If you were to generate an image according to the caption 'a red cube presses a blue cube', would you put the red or the blue cube at the bottom?
Answer: The blue cube.

If you were to generate an image according to the caption 'a red cube supports a blue cube', would you put the red or the blue cube at the bottom?
Answer: The red cu